In [1]:
from google.cloud import bigquery
import tensorflow as tf

import pandas as pd

import json
import os
from datetime import date,datetime,timedelta

import functions_framework

from google.cloud.exceptions import NotFound
from google.api_core.exceptions import BadRequest

print(tf.__version__)

2.11.0


In [2]:
# @functions_framework.http
# def predict_incident_severity_by_tf(request):

In [10]:
PROJECT_ID='smart-data-ml'

# PATH_FOLDER_ARTIFACTS="gs://tf1-incident-smart-ml-yip/model"
PATH_FOLDER_ARTIFACTS="model" 

predict_from_date=os.environ.get('predict_from_date', '')
# predict_from_date='2023-03-01'

# map_sevirity_to_class={'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


# load_model_option=os.environ.get('load_model_option', '1')
# if load_model_option=='1':
#   PATH_FOLDER_ARTIFACTS=Model_Local_Path  
# elif load_model_option=='2':
#   PATH_FOLDER_ARTIFACTS=Model_GS_Path
# else:
#   raise Exception("Allow you to set 1 for local and 2 for google storage")
# print(f"Load data from {PATH_FOLDER_ARTIFACTS}")

In [4]:
table_id = f"{PROJECT_ID}.SMartML.new_incident"
predictResult_table_id=f"{PROJECT_ID}.SMartML.new_result_prediction_incident"
unUsedColtoPredict=['severity','id','severity_id','severity_name','imported_at']

In [5]:
mapping_file="incident_sevirity_to_class.json"
with open(mapping_file, 'r') as json_file:
     map_sevirity_to_class= json.load(json_file)

print(map_sevirity_to_class)

{'Cosmatic': 0, 'Minor': 1, 'Major': 2, 'Critical': 3}


In [6]:
# Get today's date
prediction_datetime=datetime.now()

today = date.today()

# Yesterday date
if predict_from_date=='':
 yesterday = today - timedelta(days = 1)
 str_yesterday=yesterday.strftime('%Y-%m-%d')
else:
 str_yesterday=predict_from_date

str_today=today.strftime('%Y-%m-%d')

print(f"Get data between {str_yesterday} to {str_today} to predict sevirity level")

Get data between 2023-03-31 to 2023-04-01 to predict sevirity level


In [7]:
client = bigquery.Client(PROJECT_ID)
def load_data_bq(sql:str):
 
 query_result=client.query(sql)
 df=query_result.to_dataframe()
 return df

In [8]:
sql=f"""
SELECT *  FROM `{table_id}` 
 WHERE DATE(imported_at) >= '{str_yesterday}' and DATE(imported_at) < '{str_today}' 

"""
print(sql)
#LIMIT 2
dfNewData=load_data_bq(sql)
dfNewData=dfNewData.drop_duplicates(subset=['id'],keep='first')

dfNewData.insert(2, 'severity', dfNewData['severity_name'].map(map_sevirity_to_class),True)


print(dfNewData.info())
print(dfNewData)

if len(dfNewData)==0:
    print("No Data To predict")
    quit()
    #return "No Data To predict"
    


SELECT *  FROM `smart-data-ml.SMartML.new_incident` 
LIMIT 2


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   id                         2 non-null      Int64         
 1   severity_id                2 non-null      Int64         
 2   severity                   2 non-null      int64         
 3   severity_name              2 non-null      object        
 4   sla                        2 non-null      object        
 5   product_type               2 non-null      object        
 6   brand                      2 non-null      object        
 7   service_type               2 non-null      object        
 8   incident_type              2 non-null      object        
 9   open_to_close_hour         2 non-null      float64       
 10  response_to_resolved_hour  2 non-null      float64       
 11  imported_at

SyntaxError: 'return' outside function (3326159740.py, line 22)

In [11]:
try:
    model = tf.keras.models.load_model(PATH_FOLDER_ARTIFACTS)    
    print(f"Load from {PATH_FOLDER_ARTIFACTS}")
    # print(model.summary())
except Exception as error:
    
  print(str(error))
  raise error


Load from model


In [12]:
pdPrediction=pd.DataFrame(columns=['_id','predict_severity','prob_severity'])

for  row_dict in dfNewData.to_dict(orient="records"):
      incident_id=row_dict['id']
      print(f"{incident_id} - {row_dict['severity']}({row_dict['severity_name']})") 
      for key_removed in unUsedColtoPredict:
       row_dict.pop(key_removed)
      # print(row_dict)  

      input_dict = {name: tf.convert_to_tensor([value]) for name, value in row_dict.items()}


      predictionResult = model.predict(input_dict)
      result_str=','.join([ str(prob) for prob in predictionResult[0]])  
      print(result_str)   

      prob = tf.nn.softmax(predictionResult)
      prob_pct=(100 * prob)  
      _class = tf.argmax(predictionResult,-1).numpy()[0]
      
      dictPrediction={'_id':incident_id, 'predict_severity':_class,'prob_severity':result_str} 
      pdPrediction =pd.concat([pdPrediction,pd.DataFrame.from_dict([dictPrediction])] )

      print(f"{prob_pct} %   as {_class}")     
      print("======================================================================================")
            
dfPredictData=pd.merge(dfNewData,pdPrediction,how='inner',left_on='id',right_on='_id')
dfPredictData=dfPredictData.drop(columns=['_id'])
dfPredictData['predict_severity']=dfPredictData['predict_severity'].astype('int')
dfPredictData=dfPredictData[['id','prob_severity','predict_severity','severity']]
dfPredictData['prediction_item_date']= datetime.strptime(str_yesterday, '%Y-%m-%d')
dfPredictData['prediction_datetime']=prediction_datetime

1867 - 0(Cosmatic)
1/1 [==============================] - 1s 729ms/step
0.9363888,0.056718506,0.006891827,8.9424395e-07
[[45.419174 18.84531  17.92932  17.806196]] %   as 0
2391 - 2(Major)
1/1 [==============================] - 0s 69ms/step
0.0034482381,0.10939542,0.81639844,0.070757896
[[18.396084 20.45209  41.474884 19.676943]] %   as 2


In [13]:
print(dfPredictData.info())
print(dfPredictData)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2 entries, 0 to 1
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   id                    2 non-null      object        
 1   prob_severity         2 non-null      object        
 2   predict_severity      2 non-null      int32         
 3   severity              2 non-null      int64         
 4   prediction_item_date  2 non-null      datetime64[ns]
 5   prediction_datetime   2 non-null      datetime64[ns]
dtypes: datetime64[ns](2), int32(1), int64(1), object(2)
memory usage: 104.0+ bytes
None
     id                                    prob_severity  predict_severity  \
0  1867  0.9363888,0.056718506,0.006891827,8.9424395e-07                 0   
1  2391   0.0034482381,0.10939542,0.81639844,0.070757896                 2   

   severity prediction_item_date        prediction_datetime  
0         0           2023-03-31 2023-04-01 20:52:

In [14]:
#https://cloud.google.com/bigquery/docs/samples/bigquery-create-table#bigquery_create_table-python

try:
    client.get_table(predictResult_table_id)  # Make an API request.
    print("Predict Result Table {} already exists.".format(predictResult_table_id))
except Exception as ex:
    schema = [
    bigquery.SchemaField("id", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("prob_severity", "STRING", mode="REQUIRED"),
    bigquery.SchemaField("predict_severity", "INTEGER", mode="REQUIRED"),
    bigquery.SchemaField("severity", "INTEGER", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_item_date", "DATETIME", mode="REQUIRED"),    
    bigquery.SchemaField("prediction_datetime", "DATETIME", mode="REQUIRED") 
    ]

    table = bigquery.Table(predictResult_table_id,schema=schema)
    table.time_partitioning = bigquery.TimePartitioning(
    type_=bigquery.TimePartitioningType.DAY,field="prediction_item_date")
    
    table = client.create_table(table)  # Make an API request.
    
    print(
        "Created table {}.{}.{}".format(table.project, table.dataset_id, table.table_id)
    )

Predict Result Table smart-data-ml.SMartML.new_result_prediction_incident already exists.


In [15]:
def loadDataFrameToBQ():
    try:
        job_config = bigquery.LoadJobConfig(
            write_disposition="WRITE_APPEND",
        )

        job = client.load_table_from_dataframe(
            dfPredictData, predictResult_table_id, job_config=job_config
        )
        job.result()  # Wait for the job to complete.
        print("Total Prediction ML ", len(dfPredictData), "Imported bigquery successfully")

    except BadRequest as e:
        print("Bigquery Error\n")
        for e in job.errors:
            print('ERROR: {}'.format(e['message']))

try:
    loadDataFrameToBQ()
except Exception as ex:
    raise ex

Total Prediction ML  2 Imported igquery successfully


In [ ]:
# return 'ok'

In [ ]:
# if __name__ == "__main__":
#  result=predict_incident_severity_by_tf(None)
#  print(result)
